# DSC 198 Soccer GPS Data Exploration

In [1]:
import pandas as pd
import numpy as np
import math
import datetime

import plotly.graph_objects as go
import plotly.express as px
from matplotlib import pyplot as plt
import plotly
import os
import sys
sys.path.insert(0,'..')
sys.path.insert(0,'../data')

import src.viz as viz

import warnings
warnings.filterwarnings("ignore")

In [2]:
os.getcwd()

'C:\\Users\\16413\\Desktop\\DSC198-PASS-Soccer-Key-Play-Vis-Project-main\\notebooks'

In [3]:
main_folder = '../'
game_folder = '../DSC198-PASS-Soccer-Key-Play-Vis-Project/data/'

games = ['Humbolt', 'Point Loma', 'Pomona', 'Sonoma']

In [4]:
humbolt_games = game_folder + games[0]
pointloma_games = game_folder + games[1]
pomona_games = game_folder + games[2]
sonoma_games = game_folder + games[3]

In [5]:
path = os.getcwd()
print("Current Directory", path)
  
# prints parent directory
pr_path = os.path.abspath(os.path.join(path, os.pardir))
print(os.path.abspath(os.path.join(path, os.pardir)))

Current Directory C:\Users\16413\Desktop\DSC198-PASS-Soccer-Key-Play-Vis-Project-main\notebooks
C:\Users\16413\Desktop\DSC198-PASS-Soccer-Key-Play-Vis-Project-main


In [68]:
os.chdir(pr_path + '/data/Sonoma')

In [69]:
os.getcwd()

'C:\\Users\\16413\\Desktop\\DSC198-PASS-Soccer-Key-Play-Vis-Project-main\\data\\Sonoma'

In [8]:
os.chdir()

TypeError: Required argument 'path' (pos 1) not found

In [70]:
player_files = os.listdir()
player_files

['WSOC v Sonoma St Export for Amanda Erickson 3701.csv',
 'WSOC v Sonoma St Export for Ashlynn Kolarik 4247.csv',
 'WSOC v Sonoma St Export for Caitlin McCarthy 4396.csv',
 'WSOC v Sonoma St Export for Christina Oddone 4663.csv',
 'WSOC v Sonoma St Export for Danielle Satterwhite 4784.csv',
 'WSOC v Sonoma St Export for Delaney Whittet 4498.csv',
 'WSOC v Sonoma St Export for Emily Killeen 4568.csv',
 'WSOC v Sonoma St Export for Erika Braun 4796.csv',
 'WSOC v Sonoma St Export for Hana Law 4502.csv',
 'WSOC v Sonoma St Export for Kameryn Hoban 4269.csv',
 'WSOC v Sonoma St Export for Katelyn Meyer 4821.csv',
 'WSOC v Sonoma St Export for Katherine Hottinger 3914.csv',
 'WSOC v Sonoma St Export for Kelsey Kimball 4348.csv',
 'WSOC v Sonoma St Export for Lucy Tang 4564.csv',
 'WSOC v Sonoma St Export for Marissa Ray 3495.csv',
 'WSOC v Sonoma St Export for Mia Bonifazi 3471.csv',
 'WSOC v Sonoma St Export for Michelle Baddour 3966.csv',
 'WSOC v Sonoma St Export for Mikaela Celeste 4386

In [71]:
def lat_con(lat):
    #y0 = 32.887444
    #y0 = 32.887423
    y0 = 32.887890643008
    output = ((lat - y0)*100000)
    return output

def long_con(long):
    #x0 = -117.2403311
    x0 = -117.24008563915
    #x0 = -117.240433
    
    output = (long - x0)*100000
    return output

In [72]:
def convert_latitude(lat):
    x = 32.887890643008
    converted = (lat - x) * 100000
    
    return converted

def convert_longitude(long):
    y = -117.24008563915
    converted = (long - y) * 100000
    
    return converted
    
def rotate(point, radians, origin=(0, 0)):
    """Rotate a point around a given point.
    
    I call this the "low performance" version since it's recalculating
    the same values more than once [cos(radians), sin(radians), x-ox, y-oy).
    It's more readable than the next function, though.
    """
    x, y = point
    ox, oy = origin

    qx = ox + math.cos(radians) * (x - ox) + math.sin(radians) * (y - oy)
    qy = oy + -math.sin(radians) * (x - ox) + math.cos(radians) * (y - oy)

    return qx, qy

radian_val = 3.0846278020723585

In [73]:
game_df = pd.DataFrame()

for player_file in player_files:
    curr = pd.read_csv(player_file, skiprows = 8)
    
    name = ' '.join(player_file.split(' ')[-3:-1])
    curr['Player'] = name
    
    curr['Seconds'] = curr['Seconds'].apply(lambda x: int(x))
    curr = curr.groupby('Seconds').tail(1)
    
    game_df = pd.concat([game_df, curr], ignore_index = True)

In [74]:
game_df.head()

,Timestamp,Seconds,Velocity,Acceleration,Odometer,Latitude,Longitude,Heart Rate,Player Load,Positional Quality (%),HDOP,#Sats,Player
0,11/17/2019 14:51:29.99,0,0.08,0.054344,0.0,32.888011,-117.240127,0,0.0,69.0,0.78,12,Amanda Erickson
1,11/17/2019 14:51:30.99,1,0.00,-0.090675,0.0,32.888010,-117.240126,0,0.0,69.2,0.78,12,Amanda Erickson
2,11/17/2019 14:51:31.99,2,0.12,0.151896,0.1,32.888011,-117.240127,0,0.0,70.0,0.78,12,Amanda Erickson
3,11/17/2019 14:51:32.99,3,0.01,-0.129369,0.1,32.888010,-117.240126,0,0.0,70.7,0.78,12,Amanda Erickson
4,11/17/2019 14:51:33.99,4,0.00,-0.000342,0.1,32.888011,-117.240127,0,0.0,72.7,0.78,12,Amanda Erickson


In [75]:
def get_coordinates(lat, long):
    new_lat, new_long = rotate((lat, long), radian_val)
    
    return new_lat, new_long

In [76]:
def get_player_coordinates(df):
    df['Converted Latitude'] = df['Latitude'].apply(convert_latitude)
    df['Converted Longitude'] = df['Longitude'].apply(convert_longitude)
    
    coordinates = df.apply(lambda x: get_coordinates(x['Converted Latitude'],
                                                     x['Converted Longitude']),
                           axis = 1)
    
    return df

In [77]:
coord_df = get_player_coordinates(game_df)
coord_df.head()

,Timestamp,Seconds,Velocity,Acceleration,Odometer,Latitude,Longitude,Heart Rate,Player Load,Positional Quality (%),HDOP,#Sats,Player,Converted Latitude,Converted Longitude
0,11/17/2019 14:51:29.99,0,0.08,0.054344,0.0,32.888011,-117.240127,0,0.0,69.0,0.78,12,Amanda Erickson,12.035699,-4.106085
1,11/17/2019 14:51:30.99,1,0.00,-0.090675,0.0,32.888010,-117.240126,0,0.0,69.2,0.78,12,Amanda Erickson,11.965699,-4.086085
2,11/17/2019 14:51:31.99,2,0.12,0.151896,0.1,32.888011,-117.240127,0,0.0,70.0,0.78,12,Amanda Erickson,12.055699,-4.136085
3,11/17/2019 14:51:32.99,3,0.01,-0.129369,0.1,32.888010,-117.240126,0,0.0,70.7,0.78,12,Amanda Erickson,11.955699,-4.086085
4,11/17/2019 14:51:33.99,4,0.00,-0.000342,0.1,32.888011,-117.240127,0,0.0,72.7,0.78,12,Amanda Erickson,11.985699,-4.116085


In [78]:
coord_df['Converted Latitude'] = coord_df['Converted Latitude'].apply(lambda x: x + 51.5)
coord_df['Converted Longitude'] = coord_df['Converted Longitude'].apply(lambda x: x + 33)

In [79]:
barebones = coord_df[['Player', 'Seconds', 'Converted Latitude', 'Converted Longitude']]

barebones['Converted Latitude'] = barebones['Converted Latitude'].apply(lambda x: x if (x >= -1) & (x <= 105) else np.nan)
barebones['Converted Longitude'] = barebones['Converted Longitude'].apply(lambda x: x if (x >= -1) & (x <= 68) else np.nan)

barebones = barebones.dropna()
barebones.head()

,Player,Seconds,Converted Latitude,Converted Longitude
0,Amanda Erickson,0,63.535699,28.893915
1,Amanda Erickson,1,63.465699,28.913915
2,Amanda Erickson,2,63.555699,28.863915
3,Amanda Erickson,3,63.455699,28.913915
4,Amanda Erickson,4,63.485699,28.883915


# Make a shape for the defenders

In [152]:
kimball = barebones.copy()
kimball = pd.DataFrame()
for i in ['Natalie Saddic', 'Natalie Widmer', 'Kelsey Kimball', 'Delaney Whittet']:
    kimball = kimball.append(barebones[(barebones['Player'] == i)])

In [153]:
kimball.Player.unique()

array(['Natalie Saddic', 'Natalie Widmer', 'Kelsey Kimball',
       'Delaney Whittet'], dtype=object)

In [154]:
seconds = [i for i in range(0,9360,60)]

# make list of players
player_names = []
for player in kimball["Player"]:
    if player not in player_names:
        player_names.append(player)
# make figure
fig_dict = {
    "data": [],
    "layout": {},
    "frames": []
}

# fill in most of layout
fig_dict["layout"]["xaxis"] = {"range": [0, 103], "title": "Latitude"}
fig_dict["layout"]["yaxis"] = {"range": [0, 66], "title": "Longitude", }
fig_dict["layout"]["hovermode"] = "closest"


fig_dict["layout"]["updatemenus"] = [
    {
        "buttons": [
            {
                "args": [None, {"frame": {"duration": 500, "redraw": False},
                                "fromcurrent": True, "transition": {"duration": 300,
                                                                    "easing": "quadratic-in-out"}}],
                "label": "Play",
                "method": "animate"
            },
            {
                "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                  "mode": "immediate",
                                  "transition": {"duration": 0}}],
                "label": "Pause",
                "method": "animate"
            }
        ],
        "direction": "left",
        "pad": {"r": 10, "t": 87},
        "showactive": False,
        "type": "buttons",
        "x": 0.1,
        "xanchor": "right",
        "y": 0,
        "yanchor": "top"
    }
]

sliders_dict = {
    "active": 0,
    "yanchor": "top",
    "xanchor": "left",
    "currentvalue": {
        "font": {"size": 20},
        "prefix": "Second:",
        "visible": True,
        "xanchor": "right"
    },
    "transition": {"duration": 300, "easing": "cubic-in-out"},
    "pad": {"b": 10, "t": 50},
    "len": 0.9,
    "x": 0.1,
    "y": 0,
    "steps": []
}

# make data
second = 0
for player in player_names:
    player_by_second = kimball[kimball["Seconds"] == second]
    player_by_second_and_name = player_by_second[
        player_by_second["Player"] == player]
# make dictionary for the player's scatter point    
    data_dict = {
        "x": list(player_by_second["Converted Latitude"]),
        "y": list(player_by_second["Converted Longitude"]),
        "mode": "markers+text",
        "text": list(player_by_second["Player"].str.split().str.get(1)),
        'textposition':"bottom center",
        "marker_color": "darkblue",
        "marker": {
            "sizemode": "area",
            "sizeref": 200000,
            
        },
        "name": player
    }
    fig_dict["data"].append(data_dict)
    
    a = (player_by_second["Converted Latitude"]).tolist()
    b = (player_by_second["Converted Longitude"]).tolist()
# check if the player's GPS is empty(outside of the pitch)    
    if len(a) == 0:
        a = None
    if len(b) == 0:
        b = None
    else:
        a.append(a[0])
        b.append(b[0])
    
# make lines/shape that connect the players together
    line_dict = {
        "x": a,
        "y": b,
        "mode": "lines",
        'fill':"toself",
    }
    fig_dict['data'].append(line_dict)

# make frames
for second in seconds:
    frame = {"data": [], "name": str(second)}
    for player in player_names:
        player_by_second = kimball[kimball["Seconds"] == int(second)]
        player_by_second_and_name = player_by_second[
            player_by_second["Player"] == player]

        data_dict = {
        "x": list(player_by_second["Converted Latitude"]),
        "y": list(player_by_second["Converted Longitude"]),
            "mode": "markers+text",
            "text": list(player_by_second["Player"].str.split().str.get(1)),
            'textposition':"bottom center",
            "marker_color": "darkblue",
            "marker": {
                "sizemode": "area",
                "sizeref": 200000,
                
            },
            "name": player
        }
        frame["data"].append(data_dict)
        
        a = (player_by_second["Converted Latitude"]).tolist()
        b = (player_by_second["Converted Longitude"]).tolist()
        
        if len(a) == 0:
            a = None
        if len(b) == 0:
            b = None
        else:
            a.append(a[0])
            b.append(b[0])
        
        line_dict = {
            "x": a,
            "y": b,
            "mode": "lines",
            'fill':"toself",
        }
        frame['data'].append(line_dict)

    fig_dict["frames"].append(frame)
    slider_step = {"args": [
        [second],
        {"frame": {"duration": 300, "redraw": False},
         "mode": "immediate",
         "transition": {"duration": 300}}
    ],
        "label": second,
        "method": "animate"}
    sliders_dict["steps"].append(slider_step)


fig_dict["layout"]["sliders"] = [sliders_dict]


fig = go.Figure(fig_dict)


fig.update_layout(
                images= [dict(
                    source='..\\newplot2.png',
                    xref="paper", yref="paper",
                    x=0, y=0,
                    sizex=1, sizey=1,
                    xanchor="left",
                    yanchor="bottom",
                    sizing="stretch",
                    layer="below")])

fig.update_xaxes(showgrid=False, zeroline=False)
fig.update_yaxes(showgrid=False, zeroline=False)

fig.show()

# Heatmap (ongoing process)

In [151]:
# CAUTION!! This result will make your laptop lag! 
# Change the parameter in line 8 from 5000 to a very small number before running!


# player_names = 'Lucy Tang'
# shorted_barebones = barebones[(barebones['Player'] == 'Lucy Tang') 
#                               & (barebones['Seconds'] < 5000)]

# fig_dict = {"data": [], "layout": {}}

# # fill in most of layout
# fig_dict["layout"]["xaxis"] = {"range": [0, 103], "title": "Latitude"}
# fig_dict["layout"]["yaxis"] = {"range": [0, 66], "title": "Longitude", }
# fig_dict["layout"]["hovermode"] = "closest"

# player_seconds = shorted_barebones[shorted_barebones["Player"] == player_names].Seconds
# for i in player_seconds:
#     data_dict = {
#         "x": list(shorted_barebones[shorted_barebones["Seconds"] == i]["Converted Latitude"]),
#         "y": list(shorted_barebones[shorted_barebones["Seconds"] == i]["Converted Longitude"]),
#         "mode": "markers",
#         "marker_color": "darkblue",
#         "marker": {
#             "sizemode": "area",
#             "sizeref": 2000,
            
#         },
#         "name": player
#     }
#     fig_dict["data"].append(data_dict)
    
# fig = go.Figure(fig_dict)


# fig.update_layout(
#                 images= [dict(
#                     source='..\\newplot2.png',
#                     xref="paper", yref="paper",
#                     x=0, y=0,
#                     sizex=1, sizey=1,
#                     xanchor="left",
#                     yanchor="bottom",
#                     sizing="stretch",
#                     layer="below")])

# fig.update_xaxes(showgrid=False, zeroline=False)
# fig.update_yaxes(showgrid=False, zeroline=False)

# fig.show()